In [2]:
import time

from selenium import webdriver
from selenium.webdriver.chrome.webdriver import WebDriver

from openpyxl import Workbook
import pandas as pd

In [3]:
URL = "https://www.google.com/maps/place/Mie+Gacoan+Tangerang+-+Cipondoh/@-6.1876727,106.6604847,17z/data=!4m17!1m8!3m7!1s0x2e69f94f1b474fef:0x7cf44cb3839c017d!2sMie+Gacoan+Tangerang+-+Cipondoh!8m2!3d-6.1875447!4d106.6623408!10e4!16s%2Fg%2F11kbdgnjzd!3m7!1s0x2e69f94f1b474fef:0x7cf44cb3839c017d!8m2!3d-6.1875447!4d106.6623408!9m1!1b1!16s%2Fg%2F11kbdgnjzd?entry=ttu"
DriverLocation = "./Driver/chromedriver.exe"

In [23]:
options = webdriver.ChromeOptions()
#options.add_argument("--headless")  # show browser or not
options.add_argument("--lang=en-US")
options.add_experimental_option('prefs', {'intl.accept_languages': 'en,en_US'})
DriverPath = DriverLocation
driver = webdriver.Chrome(DriverPath, options=options)

driver.get(URL)
time.sleep(5)

result = driver.find_element_by_class_name('jANrlb').find_element_by_class_name('fontBodySmall').text
result = result.split(' ')
result = result[0].replace('.', '')
result = int(result)+1
result = 10
print('scrolling...')
scrollable_div = driver.find_element_by_xpath('//*[@id="QA0Szd"]/div/div/div[1]/div[2]/div/div[1]/div/div/div[2]/div[10]/div')
for i in range(result):
    scrolling = driver.execute_script(
            'document.getElementsByClassName("dS8AEf")[0].scrollTop = document.getElementsByClassName("dS8AEf")[0].scrollHeight',
            scrollable_div)
    
    more_elemet = driver.find_element_by_class_name('w8nwRe')
    more_elemet.click()

    time.sleep(5)

elements = driver.find_elements_by_class_name('jJc9Ad')
lst_data = []
for data in elements:
    name = data.find_element_by_xpath(
                '//*[@id="QA0Szd"]/div/div/div[1]/div[2]/div/div[1]/div/div/div[2]/div[9]/div[1]').get_attribute("aria-label")
    review = data.find_element_by_xpath('//*[@id="ChZDSUhNMG9nS0VJQ0FnSUNqaWFqV0xBEAE"]/span').text
    score = data.find_element_by_xpath('.//span[@class="kvMYJc"]').get_attribute("aria-label")

    lst_data.append([name, review, score])

driver.close()  


scrolling...


In [2]:
import time

from selenium import webdriver
from selenium.webdriver.chrome.webdriver import WebDriver

from openpyxl import Workbook
import pandas as pd

from env import URL, DriverLocation

In [57]:
import time

from selenium import webdriver
from selenium.common.exceptions import NoSuchElementException

import pandas as pd
from env import URL, DriverLocation

options = webdriver.ChromeOptions()
#options.add_argument("--headless")  # show browser or not
options.add_argument("--lang=en-US")
options.add_experimental_option('prefs', {'intl.accept_languages': 'en,en_US'})
DriverPath = DriverLocation
driver = webdriver.Chrome(DriverPath, options=options)
driver.set_window_size(600,700)
driver.get(URL)
driver.implicitly_wait(5)

scrollable_div = driver.find_element_by_xpath(
    '//*[@id="QA0Szd"]/div/div/div[1]/div[2]/div/div[1]/div/div/div[2]/div[10]/div')
for _i in range(5):
    scrolling = driver.execute_script(
            'document.getElementsByClassName("dS8AEf")[0].scrollTop = document.getElementsByClassName("dS8AEf")[0].scrollHeight',
            scrollable_div)
    
    more_elements = driver.find_elements_by_class_name('w8nwRe kyuRq')
    for list_more_element in more_elements:
        list_more_element.click()
    
elements = driver.find_elements_by_class_name(
        'jftiEf')
lst_data = []
for data in elements:
    try:
        name = data.find_element_by_xpath(
                './/button[@class="al6Kxe"]/div[1]').text
        text = data.find_element_by_xpath(
                './/div[@class="MyEned"]/span[1]').text
        score = data.find_element_by_xpath(
                './/span[@class="kvMYJc"]').get_attribute("aria-label")
    except NoSuchElementException:
        print("exception handled")
    lst_data.append([name, text, score])


driver.close()


cols = ["name", "comment", 'rating']
df = pd.DataFrame(lst_data, columns=cols)
df.to_csv('out.csv')
print('Done!')

Done!


In [53]:
lst_data

[['Meily Pujianti',
  'Mie nya enak. Apalagi pas pertama kali coba.  Tempat parkir luas. Tempat duduk banyak. …',
  '4 bintang'],
 ['Irfan Nabil',
  'Kadangkali kesini ketika pengen makan mie gacoan lagi. Terkadang menjadi salah satu opsi untuk bermain dengan teman teman dan adik saya. Luar biasa mie gacoan ini selain sudah setahun lebih rame terus juga memang enak baik mie dan gorengan …',
  '4 bintang'],
 ['Fadhila Fatin',
  'Cuma ya kenapa si mie gacoan kalo ngasih minuman harus nunggu orang kepedesan mulu😭 mie nya enak, pelayanan nya juga baguss, tapi minumnya selalu kelamaan datengnya, mau di gacoan manapun sama aja kok, keliatan rame banget tapi ternyata ga …',
  '5 bintang'],
 ['Indah Sulistyowati',
  'Makanan enak dan murah ya di sini. Minuman yg biasanya mahal kalo di resto lain, di sini ga mahal sama sekali. Sistem antriannya dah ok banget, pake alat portable yg akan berbunyi kalau minuman dan dimsum sdh siap diambil. Duduk sesuai …',
  '5 bintang'],
 ['puput laily',
  'Hiii!

In [54]:
dat = pd.read_csv("out.csv")
dat[dat["name"]=="Meily Pujianti"]

,Unnamed: 0,name,comment,rating
0,0,Meily Pujianti,Mie nya enak. Apalagi pas pertama kali coba. ...,4 bintang


In [63]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.action_chains import ActionChains
from confluent_kafka import Producer
import time
import socket
import json

options = webdriver.ChromeOptions()
options.add_argument('--no-sandbox')
options.add_argument('--headless')
options.add_argument('--ignore-certificate-errors')
options.add_argument('--disable-dev-shm-usage')
options.add_argument('--disable-extensions')
options.add_argument('--disable-gpu')

DriverPath = DriverLocation
driver = webdriver.Chrome(DriverPath, options=options)
class Sign:

    def  __init__(self):
        self.sign = 0

    def acked(self, err, msg):
        if err is not None:
            self.sign = 0
        else:
            self.sign = 1

conf = {'bootstrap.servers': 'localhost:29092,localhost:29093',
        'client.id': socket.gethostname()}
producer = Producer(conf)
topic_name = 'reviewTopic'

# Set the URL Target
url = "https://play.google.com/store/apps/details?id=com.whatsapp"

# Call the URL and let the browser open the web page
driver.get(url)
driver.implicitly_wait(5)
driver.set_window_size(1936, 1096)

# Find the element where "See all reviews" is exit, then click it
whatsapp_link = driver.find_elements(By.CSS_SELECTOR, "span.VfPpkd-vQzf8d")
whatsapp_link[-1].click()
time.sleep(5)

length = 0
offset = 0

while True:
    popup = driver.find_elements(By.CSS_SELECTOR, "div.RHo1pe")
    action = ActionChains(driver)
    action.move_to_element(to_element=popup[-1]).perform()
    #action.scroll_by_amount(0, 2333).perform()

    length = len(popup) - length
    review = {}
    for j in range(offset, offset + length):
        sign_obj = Sign()

        try:
            review["name"] = popup[j].find_element(By.CSS_SELECTOR, "div.X5PpBb").text
            review["date"] = popup[j].find_element(By.CSS_SELECTOR, "span.bp9Aid").text
            review["message"] = popup[j].find_element(By.CSS_SELECTOR, "div.h3YV2d").text
            review["star"] = len(popup[j].find_elements(By.CSS_SELECTOR, "span.Z1Dz7b"))
        except:
            break

        while sign_obj.sign == 0:
            producer.produce(topic_name, value=json.dumps(review).encode(), callback=sign_obj.acked)
            producer.poll(1)
        
        print("Succesfully send:", review)
    
    offset = j+1

    if length > 100000:
        break

driver.quit()

Succesfully send: {'name': 'Putu Ngurah', 'date': 'April 24, 2024', 'message': 'I find WhatsApp to be very helpful in daily communication due to its ease and efficiency. I appreciate that WhatsApp is free from intrusive ads. However, I feel that the recent UI updates in WhatsApp are not satisfactory. The previously simple and easy-to-navigate interface now feels cluttered, especially with the addition of channel and community features, which seem to disrupt the clean and simple WhatsApp experience. The new icons also appear less cohesive with the previous design.', 'star': 3}
Succesfully send: {'name': 'Sultan Muwaffaq', 'date': 'April 18, 2024', 'message': "Moving navigation to the bottom is a bit disorienting because we're so used to have space at the bottom to touch or swipe but it's still not that bad. The new color scheme for light version where everything is now white with neon green is too bright, while the dark version using black instead of dark gray too dark. You're giving us